In [24]:
import pandas as pd
import sqlite3

In [25]:
# Step 1: Read data from CSV and Excel files

df1 = pd.read_csv("C:/Users/LENOVO/Documents/jupyternotebook/Hospitalisation details.csv")
df2 = pd.read_csv("Medical Examinations.csv")
df3 = pd.read_excel("C:/Users/LENOVO/Documents/jupyternotebook/Names.xlsx")
df1.head()

,Customer ID,year,month,date,children,charges,Hospital tier,City tier,State ID
0,Id2335,1992,Jul,9,0,563.84,tier - 2,tier - 3,R1013
1,Id2334,1992,Nov,30,0,570.62,tier - 2,tier - 1,R1013
2,Id2333,1993,Jun,30,0,600.00,tier - 2,tier - 1,R1013
3,Id2332,1992,Sep,13,0,604.54,tier - 3,tier - 3,R1013
4,Id2331,1998,Jul,27,0,637.26,tier - 3,tier - 3,R1013


In [26]:
df2.head()

,Customer ID,BMI,HBA1C,Heart Issues,Any Transplants,Cancer history,NumberOfMajorSurgeries,smoker
0,Id1,47.410,7.47,No,No,No,No major surgery,yes
1,Id2,30.360,5.77,No,No,No,No major surgery,yes
2,Id3,34.485,11.87,yes,No,No,2,yes
3,Id4,38.095,6.05,No,No,No,No major surgery,yes
4,Id5,35.530,5.45,No,No,No,No major surgery,yes


In [27]:
df2['NumberOfMajorSurgeries'] = pd.to_numeric(df2['NumberOfMajorSurgeries'], errors='coerce')
df2['NumberOfMajorSurgeries'].fillna(0, inplace=True)  # Assuming default 0 surgeries
df2.head()

,Customer ID,BMI,HBA1C,Heart Issues,Any Transplants,Cancer history,NumberOfMajorSurgeries,smoker
0,Id1,47.410,7.47,No,No,No,0.0,yes
1,Id2,30.360,5.77,No,No,No,0.0,yes
2,Id3,34.485,11.87,yes,No,No,2.0,yes
3,Id4,38.095,6.05,No,No,No,0.0,yes
4,Id5,35.530,5.45,No,No,No,0.0,yes


In [28]:
df3.head()

,Customer ID,name
0,Id1,"Hawks, Ms. Kelly"
1,Id2,"Lehner, Mr. Matthew D"
2,Id3,"Lu, Mr. Phil"
3,Id4,"Osborne, Ms. Kelsey"
4,Id5,"Kadala, Ms. Kristyn"


In [29]:
# Step 2: Clean data (remove duplicates and null values)
df1.drop_duplicates(subset=['Customer ID'], inplace=True)
df1.dropna(subset=['Customer ID'], inplace=True)
df2.drop_duplicates(subset=['Customer ID'], inplace=True)
df2.dropna(subset=['Customer ID'], inplace=True)
df3.drop_duplicates(subset=['Customer ID'], inplace=True)
df3.dropna(subset=['Customer ID'], inplace=True)

In [30]:
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

df1.to_sql('Hospitalization', conn, index=False, if_exists='replace')
df2.to_sql('Medical', conn, index=False, if_exists='replace')
df3.to_sql('Names', conn, index=False, if_exists='replace')

2335

In [31]:
cursor.execute('''
CREATE TABLE hospitalizations_clean (
    Customer_ID VARCHAR,
    year INTEGER,
    month VARCHAR,
    date INTEGER,
    children INTEGER,
    charges REAL,
    Hospital_tier VARCHAR,
    City_tier VARCHAR,
    State_ID VARCHAR,
    PRIMARY KEY (Customer_ID)
)
''')

In [32]:
cursor.execute('''
INSERT INTO hospitalizations_clean
SELECT * FROM Hospitalization
''')

In [33]:
 #Create patients table with primary key
cursor.execute('''
CREATE TABLE Medical_clean (
    Customer_ID VARCHAR,
    BMI REAL,
    HBA1C REAL,
    Heart_Issues VARCHAR,
    Any_Transplants VARCHAR,
    Cancer_history VARCHAR,
    NumberOfMajorSurgeries INTEGER,
    smoker VARCHAR,
    PRIMARY KEY (Customer_ID)
)
''')

In [34]:
cursor.execute('''
INSERT INTO Medical_clean
SELECT * FROM Medical
''')

In [35]:
cursor.execute('''
CREATE TABLE Names_clean (
    Customer_ID VARCHAR,
    name VARCHAR,
    PRIMARY KEY (Customer_ID)
)
''')

In [36]:
cursor.execute('''
INSERT INTO Names_clean
SELECT * FROM Names
''')

In [37]:
query = '''
SELECT 
    h.Customer_ID,
    a.name,
    p.BMI,
    p.HBA1C,
    p.Heart_Issues,
    p.Any_Transplants,
    p.Cancer_history,
    p.NumberOfMajorSurgeries,
    p.smoker,
    h.year,
    h.month,
    h.date,
    h.children,
    h.charges,
    h.Hospital_tier,
    h.City_tier,
    h.State_ID
FROM 
    hospitalizations_clean h
JOIN 
    Medical_clean p
ON 
    h.Customer_ID = p.Customer_ID
JOIN 
    Names_clean a
ON 
    h.Customer_ID = a.Customer_ID
'''

merged_data = pd.read_sql_query(query, conn)

In [38]:

# Display the merged data
print("Merged Data:")
print(merged_data.head())

Merged Data:
  Customer_ID                                name    BMI  HBA1C Heart_Issues  \
0      Id2335                German, Mr.  Aaron K  17.58   4.51           No   
1      Id2334              Rosendahl, Mr.  Evan P  17.60   4.39           No   
2      Id2333                  Albano, Ms.  Julie  16.47   6.35           No   
3      Id2332  Riveros Gonzalez, Mr.  Juan D. Sr.  17.70   6.28           No   
4      Id2331               Brietzke, Mr.  Jordan  22.34   5.57           No   

  Any_Transplants Cancer_history  NumberOfMajorSurgeries smoker  year month  \
0              No             No                       1     No  1992   Jul   
1              No             No                       1     No  1992   Nov   
2              No            Yes                       1     No  1993   Jun   
3              No             No                       1     No  1992   Sep   
4              No             No                       1     No  1998   Jul   

   date  children  charges Hosp

In [39]:
# Query 1: Retrieve information about people who are diabetic and have heart problems
query_diabetic_heart = '''
SELECT 
    AVG(strftime('%Y', 'now') - h.year) AS average_age,
    AVG(h.children) AS average_children,
    AVG(p.BMI) AS average_BMI,
    AVG(h.charges) AS average_hospitalization_cost
FROM 
    hospitalizations_clean h
JOIN 
    Medical_clean p
ON 
    h.Customer_ID = p.Customer_ID
WHERE 
    p.HBA1C > 6.5 AND p.Heart_Issues = 'yes'
'''

diabetic_heart_info = pd.read_sql_query(query_diabetic_heart, conn)
print("\nPeople who are diabetic and have heart problems:")
print(diabetic_heart_info)


People who are diabetic and have heart problems:
   average_age  average_children  average_BMI  average_hospitalization_cost
0    63.398773          1.021472    31.337577                  16426.542577


In [40]:
# Query 2: Find the average hospitalization cost for each hospital tier and each city level
query_avg_costs = '''
SELECT 
    h.Hospital_tier,
    h.City_tier,
    AVG(h.charges) AS average_hospitalization_cost
FROM 
    hospitalizations_clean h
GROUP BY 
    h.Hospital_tier, h.City_tier
'''

In [44]:
avg_costs = pd.read_sql_query(query_avg_costs, conn)
print("\nAverage hospitalization cost for each hospital tier and city level:")
print(avg_costs)


Average hospitalization cost for each hospital tier and city level:
   Hospital_tier City_tier  average_hospitalization_cost
0              ?  tier - 3                    700.000000
1       tier - 1  tier - 1                  29519.600814
2       tier - 1  tier - 2                  28788.457477
3       tier - 1  tier - 3                  31915.436786
4       tier - 2  tier - 1                  11508.067079
5       tier - 2  tier - 2                  11992.427083
6       tier - 2  tier - 3                  12093.369163
7       tier - 3         ?                    770.380000
8       tier - 3  tier - 1                   9739.840905
9       tier - 3  tier - 2                   9283.427477
10      tier - 3  tier - 3                   9342.179912


In [42]:
# Query 3: Determine the number of people who have had major surgery with a history of cancer
query_major_surgery_cancer = '''
SELECT 
    COUNT(*) AS num_people_major_surgery_with_cancer
FROM 
    Medical_clean p
WHERE 
    p.Cancer_history = 'yes' AND p.NumberOfMajorSurgeries > 0
'''

major_surgery_cancer_count = pd.read_sql_query(query_major_surgery_cancer, conn)
print("\nNumber of people who have had major surgery with a history of cancer:")
print(major_surgery_cancer_count)



Number of people who have had major surgery with a history of cancer:
   num_people_major_surgery_with_cancer
0                                     0


In [45]:
query_tier1_hospitals = '''
SELECT 
    h.State_ID,
    COUNT(*) AS num_tier1_hospitals
FROM 
    hospitalizations_clean h
WHERE 
    h.Hospital_tier = "tier - 1"
GROUP BY 
    h.State_ID
'''

tier1_hospitals_count = pd.read_sql_query(query_tier1_hospitals, conn)
print("\nNumber of tier-1 hospitals in each state:")
print(tier1_hospitals_count)




Number of tier-1 hospitals in each state:
   State_ID  num_tier1_hospitals
0         ?                    2
1     R1011                  116
2     R1012                   63
3     R1013                   68
4     R1014                   10
5     R1015                    2
6     R1016                    8
7     R1017                    7
8     R1018                    1
9     R1019                    5
10    R1023                    4
11    R1024                   14
12    R1026                    5


In [46]:
conn.close()